<i><b>Public-AI</b></i>
### ✎&nbsp;&nbsp;week 6. Neural Collaborative Filtering와 DeepFM
# Section 1. Neural Collaborative Filtering

이전 주차에 우리는 Matrix Factorization을 배웠습니다. Matrix Factorization하는 방법으로 Bayesian Personalized Ranking을 이용하여 고객에 대한 Embedding 정보, 아이템에 대한 Embedding 정보를 획득하는 방법을 배웠습니다. 이번 시간에는 Deep Learning을 활용하여 Matrix Factorization을 하는 Neural Collaborative Filtering을 배워보도록 하겠습니다.

### _Objective_ 

* Tensorflow로 Bayesian Personalized Ranking을 구성하는 방법을 배워봅니다.
* Tensorflow로 Neural Collaborative Filtering을 구성하는 방법을 배워봅니다.

In [ ]:
%matplotlib inline
import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.utils import get_file
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Embedding
np.set_printoptions(5,)

# \[ 데이터 셋 : MovieLens 100K\]
---


## 1. 데이터 셋 가져오기

In [ ]:
ROOT_URL = "https://pai-datasets.s3.ap-northeast-2.amazonaws.com/recommender_systems/movielens_100k/datasets/"

# 데이터 가져오기
ratings_path = get_file("100k_ratings.csv", ROOT_URL+"ratings.csv")
movies_path = get_file("100k_movies.csv",ROOT_URL+"movies.csv")
users_path = get_file("100k_users.csv", ROOT_URL+"users.csv")

ratings_df = pd.read_csv(ratings_path)
movies_df = pd.read_csv(movies_path)
users_df = pd.read_csv(users_path)

## 2. 데이터 확인하기

### (1) ratings_df 데이터셋
+ user_id : user 식별자
+ item_id : 영화(Item) 식별자
+ rating : 각 user별 영화의 평가 점수

In [ ]:
print("ratings_df의 크기 : ", ratings_df.shape)

# 다섯개 데이터를 Random으로 가져옴
ratings_df.sample(5, random_state=1)

### (2) movies_df 데이터셋

+ item_id : 영화(item) 식별자
+ title : 영화(item) 제목
+ year : 영화 개봉 년도
+ unknown ~ Western : 영화 장르의 멀티핫인코딩(Multi-hot encoding)

In [ ]:
print("movies_df의 크기 : ", movies_df.shape)

# 5개를 Random으로 가져옴
movies_df.sample(5, random_state=1)

### (3) users_df 데이터셋

+ user_id : user 식별자
+ age : user의 나이 (0\~4세 :0, 5\~9세:1, ... )
+ gender : user의 성별
+ occupation : user의 직업군

In [ ]:
print("users_df의 크기 : ", users_df.shape)

# 5개를 Random으로 가져옴
users_df.sample(5, random_state=1)

## 3. K-Core Sampling을 통해 user_id, item_id 추려내기

k=5를 기준으로, 너무나 적게 평가하거나 평가받은 유저 및 영화는 제거하도록 하겠습니다.

In [ ]:
like_df = ratings_df.copy()

threshold = 5

count = 0
while True:
    prev_total = len(like_df)
    print(f"{count}회차 데이터 수 : {prev_total:,}개")
    
    total_user_per_item = (
        like_df
        .groupby('item_id')['user_id']
        .count())
    over_item_ids = total_user_per_item[
        total_user_per_item>threshold].index
    
    total_item_per_user = (
        like_df
        .groupby('user_id')['item_id']
        .count())
    over_user_ids = total_item_per_user[
        total_item_per_user>threshold].index
    
    like_df = like_df[
        (like_df.user_id.isin(over_user_ids))
        &(like_df.item_id.isin(over_item_ids))]

    if prev_total == len(like_df):
        print("종료")
        break
    count += 1

## 4. 평가 기준 설정하기

Bayesian Personalized Ranking과 Neural Collaborative Filtering의 성능을 비교해보기 위해 우리는 Hit Ratio를 이용하도록 하겠습니다.

### (1) 평가데이터셋 구성하기

우선 Train 데이터셋와 Test 데이터셋을 나누도록 하겠습니다. Test 데이터셋은 각 고객이 평가한 마지막 영화로 두도록 하겠습니다.

In [ ]:
trains = []
tests = []
for i, group_df in like_df.groupby('user_id'):
    # 마지막 직전은 Train_, 미자믹은 test_
    train_, test_ = group_df.iloc[:-1], group_df.iloc[-1:]
    trains.append(train_)
    tests.append(test_)
    
train_df = pd.concat(trains)
test_df = pd.concat(tests)

# user_id를 기준으로 정렬된 것을 무작위로 섞음
train_df = train_df.sample(frac=1)
test_df = test_df.sample(frac=1)

In [ ]:
print("train 데이터 셋의 수 : ", len(train_df))
print("test 데이터 셋의 수 : ", len(test_df))

### (2) Hit Ratio를 위한 데이터셋 구성하기 

이번 시간에는 평가지표를 Hit Ratio를 이용해보도록 하겠습니다. Hit Ratio의 측정방법은 아래와 같습니다.

> 고객이 구매한 아이템 1개와 고객이 구매하지 않은 아이템 100개를 가져온 후, 고객이 구매한 아이템 고객이 구매한 아이템이 101개 중 몇번째에 위치하는지를 확인하기. Top-10 Hit Ratio란, 고객이 구매한 아이템이 10번째 안에 들어있는 확률로, 높을수록 보다 정확하게 추천한다고 판단

In [ ]:
# 유저별 평가한 영화목록 구성하기
itemset_per_user = (
    ratings_df
    .groupby('user_id')
    ['item_id']
    .apply(frozenset)
)

total_items = set(ratings_df.item_id.unique())

# 유저가 평가하지 않은 영화목록 구성하기
notseen_itemset_per_user = total_items - itemset_per_user
notseen_itemset_per_user = notseen_itemset_per_user.apply(list)

hit ratio를 계산하기 위해서 우리는 본 영화 1개와 보지 않은 영화 100개를 구성해야 합니다. 그리고 본 영화와 보지 않은 영화 모두 모델로 추론한 후, 선호도 순서대로 정렬 후 본 영화가 10등 안에 들었으면, 모델이 올바르게 추론했다고 평가하고, 들지 않으면 모델이 잘못 추론했다고 평가하는 방식입니다.

In [ ]:
hit_ratio_df = test_df.copy()

hit_ratio_df['not_seen_list'] = hit_ratio_df.user_id.apply(
    lambda x : random.choices(notseen_itemset_per_user[x],k=100))

hit_ratio_df = hit_ratio_df.drop('rating',axis=1)
hit_ratio_df.head()

# \[ 2. Neural Collaborative Filtering 구성하기 \]
---

BPR을 이용한 Matrix Factorization은 효과적이나, 고객의 취향 행렬과 제품의 특성 행렬 간의 선형 관계만을 파악할 수 있다는 점에서 한계가 있습니다. 좀 더 복잡한 고객의 취향 <-> 특성 행렬 간 관계를 파악하기 위해 딥러닝을 Matrix Factorization에 적용하는 연구가 진행되었습니다. 이 중 대표적인 모델이 바로 아래 소개된 `Neural Collaborative Filtering`입니다.

* Code Reference : [Paper's Implementation](https://github.com/hexiangnan/neural_collaborative_filtering/blob/master/MLP.py)

## 1. 모델링하기

<img src="https://i.imgur.com/B0lLZEg.png" width="400">

Neural Collaborative Filtering은 이 구조도로 모델링을 설명할 수 있습니다. 유저의 임베딩 값과 아이템의 임베딩 값을 딥러닝의 Input으로 들어가게 됩니다. 그리고 4층을 걸쳐 출력값으로 고객이 해당 아이템을 선호할 확률이 나오게 됩니다.

### (1) 입력 구성하기

우선 user와 item이 쌍으로 Input으로 들어가게 됩니다.

In [ ]:
from tensorflow.keras.layers import Input

num_user = ratings_df.user_id.max() + 1
num_item = ratings_df.item_id.max() + 1

user_id = Input(shape=(), name='user')
item_id = Input(shape=(), name='item')

### (2) 임베딩 구성하기

임베딩의 크기는 논문에서 알려져 있듯, 값이 커질수록 그 성능이 올라갑니다. 여기에서는 32로 두도록 하겠습니다.

In [ ]:
num_factor = 32

user_embedding = Embedding(num_user, num_factor)(user_id)
item_embedding = Embedding(num_item, num_factor)(item_id)

### (3) NCF Layers 구성하기

`Dense` 레이어가 순서대로 적층되어 있는 단순한 구조를 띕니다. 논문에서 서술되어 있는 방식으로, 3층 구조로 unit수가 ( 32 $\rightarrow$ 16 $\rightarrow$ 8 )로 줄어드는 순서로 작성하였습니다.

In [ ]:
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dense

concat_embedding = Concatenate()([user_embedding, item_embedding])

hidden1 = Dense(64, activation='relu',
                kernel_regularizer=l2(1e-4),
                name='layer1')(concat_embedding)
hidden2 = Dense(32, activation='relu',
                kernel_regularizer=l2(1e-4),                
                name='layer2')(hidden1)
hidden3 = Dense(16, activation='relu',
                kernel_regularizer=l2(1e-4),                
                name='layer3')(hidden2)
probs = Dense(1, activation='sigmoid',
                kernel_regularizer=l2(1e-4),              
               name='probs')(hidden3)

### (4) Model 구성하기

입력값은 `user_id`와 `item_id`가 Pair로 들어가게 되고, 출력값은 `user_id`가 `item_id`를 선호할 확률이 계산됩니다. 

`user_id`가 `item_id`를 구매하거나 산 적이 있으면 1이 나와야 하고,
`user_id`가 `item_id`를 구매하거나 산 적이 없다면 0이 나아야 합니다.


In [ ]:
from tensorflow.keras.models import Model

model = Model([user_id, item_id], probs, name='NCF')

논문에서 학습하기 위해 `Adam` 옵티마이저를 이용하였고, 손실함수로서는 `BinaryCrossentropy`를 이용하였습니다.

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy

model.compile(Adam(1e-3), 
              loss=BinaryCrossentropy(),
              metrics=[BinaryAccuracy()])

## 2. 데이터 구성하기

Bayesian Personalized Ranking과 마찬가지로, 데이터는 오로지 고객이 평가한 영화에 대한 정보만 존재합니다. 모델을 학습하기 위해서, 고객이 평가하지 않은 영화에 대한 정보 또한 필요합니다. 보지 않은 영화 모두를 가져오게 되면 지나치게 많은 데이터가 생기므로 그중에서 일부분만 샘플링을 통해 추출하게 됩니다.

### (1) 고객이 평가하지 않은, 구매하지 않은 영화군 정의하기

Bayesian Personalized Ranking 때와 동일한 코드입니다.

In [ ]:
# 유저별 평가한 영화목록 구성하기
itemset_per_user = (
    train_df
    .groupby('user_id')
    ['item_id']
    .apply(frozenset)
)

total_items = set(train_df.item_id.unique())

# 유저가 평가하지 않은 영화목록 구성하기
notseen_itemset_per_user = total_items - itemset_per_user
notseen_itemset_per_user = notseen_itemset_per_user.apply(list)

### (2) 학습 데이터 구성하기

NCF 모델은 user, item의 쌍으로 추론합니다. 해당 User가 해당 Item을 선호할 확률을 추론하기 때문에, 실제로 클릭 혹은 구매한 적이 있으면 학습 데이터의 라벨로는 1을 주고 클릭한적이 없다면 학습 데이터의 라벨로는 0을 줍니다. 유저가 클릭하지 않은 아이템을 Negative Sample이라 하고, 우리는 Negative Sample을 만들어주어야 합니다. <br>
NCF 모델에서 Positive Sample 대비 Negative Sampling의 비율을 보통 3.~6. 사이로 두는 것을 권고합니다.

In [ ]:
def get_ncf_dataset(train_df, notseen_itemset_per_user,negative_ratio=4.):
    # 모든 rating은 기본적으로 Positive Sample
    positive_samples = train_df.copy()    
    
    # Negative Sampling 수행하기
    negative_samples = (train_df
                        .sample(frac=negative_ratio, replace=True)
                        .copy())
    negative_samples['item_id'] = negative_samples.user_id.apply(
        lambda x : random.choice(notseen_itemset_per_user[x]))
    
    # Positive와 Negative의 라벨을 지정해주기
    positive_samples['label'] = True    
    negative_samples['label'] = False
    batch_df = pd.concat([positive_samples, 
                          negative_samples])
    # 순서를 무작위로 섞기
    batch_df = batch_df.sample(frac=1)
    
    X = {
        "user":batch_df['user_id'].values,
        "item":batch_df['item_id'].values,
    }
    y = batch_df['label'].values
    
    return X, y

### (3) 모델 학습하기

epoch 10번에 걸쳐, 모델을 학습시키도록 하겠습니다. 매 Epoch마다 새로운 학습 데이터를 생성하도록 하였습니다. 

In [ ]:
epoch = 10
for i in range(1, epoch+1):
    print("{}th".format(i))
    X,y = get_ncf_dataset(train_df, notseen_itemset_per_user)
    model.fit(X, y, batch_size=1024, verbose=2)

## 3. 모델 평가하기

Bayesian Personalized Ranking 대비 성능을 비교하기 위해 Hit Ratio를 산출해보도록 하겠습니다.

In [ ]:
hit = 0.
for i, row in tqdm(hit_ratio_df.iterrows()):
    user = np.array([row.user_id])
    seens = np.array([row.item_id])
    pos_scores = model.predict([user,seens])
    pos_scores = pos_scores[0,0]
    
    not_seens = np.array(row.not_seen_list)
    users = np.array([row.user_id]*len(not_seens))   
    neg_scores = model.predict([users,not_seens])
    
    if pos_scores > np.sort(neg_scores.flatten())[-10]:
        hit += 1

In [ ]:
hit_ratio = hit / len(hit_ratio_df)        
print(f"hit ratio : {hit_ratio:.3f}")

<hr>
<div style = "background-image: url('https://pai-picture.s3.ap-northeast-2.amazonaws.com/PAI-Identity/PublicAI+Logo.png');background-repeat: no-repeat; background-position: right; background-size: 60px 40px; padding : 5px 70px 5px 5px;">
    Copyright(c) 2020 by Public AI. All rights reserved.<br>
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2020/02/18
</div>
<hr>